# Advanced Data Wrangling & Analysis

## Lesson Overview

This workshop transforms basic Python knowledge into professional data manipulation skills. We follow the "Extract-Transform-Analyze" workflow across 4 distinct sections:

1.  **Part 1: Financial Time Series & Window Functions**
    * Handling Datetime objects and indexing
    * Resampling and Frequency conversion
    * Window functions (Rolling means)
    * Covariance and Correlation
2.  **Part 2: Data Wrangling (Merge & Reshape)**
    * Merging datasets (Inner, Outer, Left, Right joins)
    * Reshaping data: Melt and Pivot
3.  **Part 3: Aggregation & Reporting**
    * GroupBy mechanics (Split-Apply-Combine)
    * Pivot Tables and Cross-Tabulations
4.  **Part 4: Advanced Toolkit (Optional/Deep Dive)**
    * Hierarchical Indexing (MultiIndex)
    * Concatenation
    * Stacking/Unstacking
    * Advanced GroupBy: Apply and Transform

---

**Setup:** Import necessary libraries.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# Part 1: Financial Time Series & Window Functions

**Learning Objective:** Handle data where the *order* matters (Time Series). We will learn to convert strings to dates, handle missing business days, smoothing volatile data, and analyzing stock correlations.

## 1.1 Handling Date Time Data

Pandas is oriented towards working with arrays of dates, whether used as an axis index or a column.

The `to_datetime` method parses may different kinds of date representations:

In [2]:
dates = ["2011-07-06 12:00:00", "2011-08-06 00:00:00"]

pd.to_datetime(dates)

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00'], dtype='datetime64[ns]', freq=None)

It uses `NaT` (Not a Time) as null values for datetime data.

In [3]:
idx = pd.to_datetime(dates + [None])
idx

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00', 'NaT'], dtype='datetime64[ns]', freq=None)

In [4]:
pd.isna(idx)

array([False, False,  True])

Standard Python uses the `datetime` module to handle date and time data. Pandas has a `Timestamp` object that is similar to the `datetime` object. 

If you use `datetime` objects as index to a Series or DataFrame, Pandas will automatically convert them to `DatetimeIndex` objects.

In [5]:
dates = [datetime(2011, 1, 2), datetime(2011, 1, 5), datetime(2011, 1, 7), 
         datetime(2011, 1, 8), datetime(2011, 1, 10), datetime(2011, 1, 12)]

ts = pd.Series(np.random.standard_normal(6), index=dates)
ts

2011-01-02    0.201869
2011-01-05    1.014364
2011-01-07    0.805327
2011-01-08    1.906294
2011-01-10    1.835678
2011-01-12    0.001200
dtype: float64

In [ ]:
# .index attribute
ts.index

DatetimeIndex(['2011-01-02', '2011-01-05', '2011-01-07', '2011-01-08',
               '2011-01-10', '2011-01-12'],
              dtype='datetime64[ns]', freq=None)

Like other Series, arithmetic operations between differently indexed time series automatically align on the dates:

In [98]:
ts[::2]

2011-01-02    0.201869
2011-01-07    0.805327
2011-01-10    1.835678
dtype: float64

In [ ]:
# [::2] selects every second element --> returns NaN when there's nothing to add
ts + ts[::2]

2011-01-02    0.403737
2011-01-05         NaN
2011-01-07    1.610654
2011-01-08         NaN
2011-01-10    3.671355
2011-01-12         NaN
dtype: float64

### Indexing & Slicing

You can index by passing a `datetime`, `Timestamp` or `string` that is interpretable as a date:

In [ ]:
print(type(datetime(2011, 1, 7)))
ts[datetime(2011, 1, 7)]


<class 'datetime.datetime'>


0.8053269460542957

In [110]:
print(type(pd.Timestamp("2011-01-07")))
ts[pd.Timestamp("2011-01-07")]

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


0.8053269460542957

In [112]:
print(type("2011-01-07"))
ts["2011-01-07"]

<class 'str'>


0.8053269460542957

You can even specify the year or year-month strings to slice a range of data. This is very powerful for quick analysis.

In [ ]:
# date_range generate an array of dates, length of period 1000days
longer_ts = pd.Series(np.random.standard_normal(1000), 
                      index=pd.date_range("2000-01-01", periods=1000))

longer_ts

2000-01-01    1.288685
2000-01-02   -0.277958
2000-01-03   -2.082372
2000-01-04   -0.729645
2000-01-05    1.860146
                ...   
2002-09-22   -1.435695
2002-09-23   -0.324766
2002-09-24   -0.340944
2002-09-25   -0.640145
2002-09-26   -1.074065
Freq: D, Length: 1000, dtype: float64

In [12]:
# Select all data from 2001
longer_ts["2001"].head()

2001-01-01   -2.440270
2001-01-02   -0.976414
2001-01-03   -0.869335
2001-01-04   -0.648844
2001-01-05   -0.174685
Freq: D, dtype: float64

In [ ]:
# Select all data in May 2001
longer_ts["2001-05"].head()

2001-05-01    1.773847
2001-05-02   -2.555258
2001-05-03   -0.699149
2001-05-04    0.941951
2001-05-05    0.684027
Freq: D, dtype: float64

In [ ]:
# Slicing with strings - select ALL FROM May 2001 onwards
longer_ts["2001-05":].head()

2001-05-01    1.773847
2001-05-02   -2.555258
2001-05-03   -0.699149
2001-05-04    0.941951
2001-05-05    0.684027
Freq: D, dtype: float64

**Student Exercise:** Use `date_range` to generate a Series of random values from 1-31st January 2023. Then slice the Series to return data from 5-15th January.

In [ ]:
jan = pd.Series(np.random.standard_normal(31), index=pd.date_range("2023-01-01", periods=31))
#jan
jan['2023-01-05':'2023-01-15']

2023-01-05   -2.544853
2023-01-06   -0.091502
2023-01-07    0.088040
2023-01-08    0.803291
2023-01-09   -2.588778
2023-01-10   -1.990518
2023-01-11   -1.809511
2023-01-12   -0.722285
2023-01-13   -0.151626
2023-01-14    0.665081
2023-01-15    0.026610
Freq: D, dtype: float64

## 1.2 Real-World Application: Stock Market Data

Let's load stock prices (AAPL, GOOG, IBM, MSFT) and trade volumes. We use `.read_pickle` here as it preserves the datetime objects native to the file.

In [ ]:
# pickle python obj for long term storage (only open pkl file u trust!)
price = pd.read_pickle("../data/yahoo_price.pkl")
volume = pd.read_pickle("../data/yahoo_volume.pkl")

In [16]:
price.head()

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-04,27.990226,313.062468,113.304536,25.884104
2010-01-05,28.038618,311.683844,111.935822,25.892466
2010-01-06,27.592626,303.826685,111.208683,25.733566
2010-01-07,27.541619,296.753749,110.823732,25.465944
2010-01-08,27.724725,300.709808,111.935822,25.641571


In [17]:
volume.head()

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-04,123432400,3927000,6155300,38409100
2010-01-05,150476200,6031900,6841400,49749600
2010-01-06,138040000,7987100,5605300,58182400
2010-01-07,119282800,12876600,5840600,50559700
2010-01-08,111902700,9483900,4197200,51197400


### Inspecting the Index
Notice the index is a `DatetimeIndex`.

In [18]:
price.index

DatetimeIndex(['2010-01-04', '2010-01-05', '2010-01-06', '2010-01-07',
               '2010-01-08', '2010-01-11', '2010-01-12', '2010-01-13',
               '2010-01-14', '2010-01-15',
               ...
               '2016-10-10', '2016-10-11', '2016-10-12', '2016-10-13',
               '2016-10-14', '2016-10-17', '2016-10-18', '2016-10-19',
               '2016-10-20', '2016-10-21'],
              dtype='datetime64[ns]', name='Date', length=1714, freq=None)

We can access attributes like `day_of_week` directly:

In [ ]:
# monday 0, sunday 6
price.index.day_of_week

Int64Index([0, 1, 2, 3, 4, 0, 1, 2, 3, 4,
            ...
            0, 1, 2, 3, 4, 0, 1, 2, 3, 4],
           dtype='int64', name='Date', length=1714)

In [20]:
price.index.month

Int64Index([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
            ...
            10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
           dtype='int64', name='Date', length=1714)

If the datetime is in a column instead of the index, you can use the `dt` accessor to access the datetime properties.

In [21]:
price_reindex = price.reset_index()
price_reindex.head()

,Date,AAPL,GOOG,IBM,MSFT
0,2010-01-04,27.990226,313.062468,113.304536,25.884104
1,2010-01-05,28.038618,311.683844,111.935822,25.892466
2,2010-01-06,27.592626,303.826685,111.208683,25.733566
3,2010-01-07,27.541619,296.753749,110.823732,25.465944
4,2010-01-08,27.724725,300.709808,111.935822,25.641571


In [ ]:
# dt to access datetime properties
price_reindex["Date"].dt.day_name().head()

0       Monday
1      Tuesday
2    Wednesday
3     Thursday
4       Friday
Name: Date, dtype: object

**Student Exercise:** Get the week of year from the date column and create a new column `week_of_year`.

In [ ]:
#price_reindex['week_of_year'] = price_reindex["Date"].dt.weekofyear -- weekofyear to be deprecated
price_reindex['week_of_year'] = price_reindex["Date"].dt.isocalendar().week 
price_reindex

,Date,AAPL,GOOG,IBM,MSFT,week_of_year
0,2010-01-04,27.990226,313.062468,113.304536,25.884104,1
1,2010-01-05,28.038618,311.683844,111.935822,25.892466,1
2,2010-01-06,27.592626,303.826685,111.208683,25.733566,1
3,2010-01-07,27.541619,296.753749,110.823732,25.465944,1
4,2010-01-08,27.724725,300.709808,111.935822,25.641571,1
...,...,...,...,...,...,...
1709,2016-10-17,117.550003,779.960022,154.770004,57.220001,42
1710,2016-10-18,117.470001,795.260010,150.720001,57.660000,42
1711,2016-10-19,117.120003,801.500000,151.259995,57.529999,42
1712,2016-10-20,117.059998,796.969971,151.520004,57.250000,42


## 1.3 Resampling (Frequency Conversion)

As you can see from above, the dates are on business days. If you want to change the frequency to calendar days (e.g. including weekends), we use `resample`.

This introduces missing data (NaN) for weekends.

In [124]:
#  asfreq() method changes freq of time series data to the specified freq 'D' daily
price_resampled = price.resample('D').asfreq()
price_resampled.head(10)

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-04,27.990226,313.062468,113.304536,25.884104
2010-01-05,28.038618,311.683844,111.935822,25.892466
2010-01-06,27.592626,303.826685,111.208683,25.733566
2010-01-07,27.541619,296.753749,110.823732,25.465944
2010-01-08,27.724725,300.709808,111.935822,25.641571
2010-01-09,NaN,NaN,NaN,NaN
2010-01-10,NaN,NaN,NaN,NaN
2010-01-11,27.480148,300.255255,110.763844,25.315406
2010-01-12,27.167562,294.945572,111.644958,25.148142


**Forward Fill (`ffill`)**: If you want to fill the na values with the most recent value (common in finance - Saturday price is same as Friday close), you can use the `.ffill()` method.

In [24]:
price_resampled = price.resample('D').ffill()
price_resampled.head(10)

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-04,27.990226,313.062468,113.304536,25.884104
2010-01-05,28.038618,311.683844,111.935822,25.892466
2010-01-06,27.592626,303.826685,111.208683,25.733566
2010-01-07,27.541619,296.753749,110.823732,25.465944
2010-01-08,27.724725,300.709808,111.935822,25.641571
2010-01-09,27.724725,300.709808,111.935822,25.641571
2010-01-10,27.724725,300.709808,111.935822,25.641571
2010-01-11,27.480148,300.255255,110.763844,25.315406
2010-01-12,27.167562,294.945572,111.644958,25.148142


If you want to resample to a **lower frequency** (e.g. Monthly 'MS' - Month Start) you need to provide an aggregation method (like `mean`):

In [25]:
price_resampled = price.resample('MS').mean()
price_resampled.head()

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-01,27.166942,289.012006,110.364037,25.212407
2010-02-01,26.000370,267.179857,107.496919,23.767984
2010-03-01,29.219760,280.235245,109.849295,24.584058
2010-04-01,32.847556,278.253415,111.324726,25.646245
2010-05-01,32.888483,248.418509,109.690449,23.670255


**Student Exercise:** Resample price to `yearly` (start of year) frequency, use `sum` as aggregation function.

In [ ]:
# 'Y' year end 31/12, 'YS' year start 1/1
price_resampled = price.resample('YS').sum()
price_resampled

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-01,8564.125188,67421.204377,28722.464346,5757.023922
2011-01-01,11997.187091,71619.248624,37895.630612,5675.814809
2012-01-01,18880.993859,80271.925770,43947.570175,6621.497457
2013-01-01,15929.515474,111303.472578,44524.331233,7498.810042
2014-01-01,22293.854902,141458.282752,42727.132433,10081.875149
2015-01-01,29501.178958,151820.449636,37431.251170,11389.086467
2016-01-01,20810.039594,150038.480286,29879.087154,10904.541111


## 1.4 Window Functions (Moving Averages)

You can apply functions evaluated over a sliding window using the `rolling` method.

For example, to compute the 30-day moving average for Apple price:

In [ ]:
# rolling(last how many days u want). (30) last 30days
price["AAPL"].rolling(30).mean().tail()

Date
2016-10-17    112.730333
2016-10-18    113.056000
2016-10-19    113.348000
2016-10-20    113.732666
2016-10-21    114.181667
Name: AAPL, dtype: float64

By default, rolling functions require all of the values in the window to be non-NA. This behavior can be changed to account for missing data and, especially at the beginning of the time series.

In [ ]:
print(price["AAPL"])    # shows values from 4Jan
# min_periods = use minimally 3 days to compute mean
price["AAPL"].rolling(30, min_periods=3).mean().head()

Date
2010-01-04     27.990226
2010-01-05     28.038618
2010-01-06     27.592626
2010-01-07     27.541619
2010-01-08     27.724725
                 ...    
2016-10-17    117.550003
2016-10-18    117.470001
2016-10-19    117.120003
2016-10-20    117.059998
2016-10-21    116.599998
Name: AAPL, Length: 1714, dtype: float64


Date
2010-01-04          NaN
2010-01-05          NaN
2010-01-06    27.873823
2010-01-07    27.790772
2010-01-08    27.777563
Name: AAPL, dtype: float64

**Student Exercise:** Compute a 10-day moving average for `GOOG` with a min period of 5 days.

In [127]:
print(price["GOOG"])
price["GOOG"].rolling(10, min_periods=5).mean()

Date
2010-01-04    313.062468
2010-01-05    311.683844
2010-01-06    303.826685
2010-01-07    296.753749
2010-01-08    300.709808
                 ...    
2016-10-17    779.960022
2016-10-18    795.260010
2016-10-19    801.500000
2016-10-20    796.969971
2016-10-21    799.369995
Name: GOOG, Length: 1714, dtype: float64


Date
2010-01-04           NaN
2010-01-05           NaN
2010-01-06           NaN
2010-01-07           NaN
2010-01-08    305.207311
                 ...    
2016-10-17    779.667004
2016-10-18    781.550006
2016-10-19    784.053009
2016-10-20    786.064007
2016-10-21    788.493005
Name: GOOG, Length: 1714, dtype: float64

## 1.5 Covariance and Correlation

Covariance and correlation measure the relationship between two variables.

* **Covariance:** Measure of how much two random variables vary together. Hard to interpret magnitude.
* **Correlation:** Normalized measure (-1 to 1). 1 is perfect positive correlation, -1 is perfect negative.

In finance, we usually look at **Returns** (Percent Change), not raw prices.

In [ ]:
# % change from the previous day (-0.000680 = -0.6%)
returns = price.pct_change()
returns.tail()

,AAPL,GOOG,IBM,MSFT
Date,,,,
2016-10-17,-0.000680,0.001837,0.002072,-0.003483
2016-10-18,-0.000681,0.019616,-0.026168,0.007690
2016-10-19,-0.002979,0.007846,0.003583,-0.002255
2016-10-20,-0.000512,-0.005652,0.001719,-0.004867
2016-10-21,-0.003930,0.003011,-0.012474,0.042096


Compute the correlation and covariance between the returns of `MSFT` and `IBM`:

In [ ]:
# stength of association/relationshiop depends on industry
print("Covariance:", returns["MSFT"].cov(returns["IBM"]))
print("Correlation:", returns["MSFT"].corr(returns["IBM"]))

Covariance: 8.870655479703546e-05
Correlation: 0.4997636114415114


You can also get the full (pair-wise) correlation or covariance matrix as a DataFrame:

In [ ]:
# correlation coefficients
returns.corr()

,AAPL,GOOG,IBM,MSFT
AAPL,1.000000,0.407919,0.386817,0.389695
GOOG,0.407919,1.000000,0.405099,0.465919
IBM,0.386817,0.405099,1.000000,0.499764
MSFT,0.389695,0.465919,0.499764,1.000000


You can also compute pair-wise correlations between a DataFrame’s columns or rows with another Series or DataFrame.

In [ ]:
# Correlation of all companies against IBM - correlation coefficient
returns.corrwith(returns["IBM"])

AAPL    0.386817
GOOG    0.405099
IBM     1.000000
MSFT    0.499764
dtype: float64

In [32]:
# Correlation of returns against volume
returns.corrwith(volume)

AAPL   -0.075565
GOOG   -0.007067
IBM    -0.204849
MSFT   -0.092950
dtype: float64

---
# Part 2: Data Wrangling (Merge & Reshape)

**Learning Objective:** Combine data from different sources (SQL-style Joins) and reshape table layouts (Wide to Long) to prepare for analysis.

## 2.1 Merging (Joins)

`merge` connects rows in DataFrames based on one or more keys. This is equivalent to database `join` operations.

In [33]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"], 
                    "data1": pd.Series(range(7), dtype="Int64")})

df2 = pd.DataFrame({"key": ["a", "b", "d"], 
                    "data2": pd.Series(range(3), dtype="Int64")})

print("DF1 (Left):\n", df1)
print("\nDF2 (Right):\n", df2)

DF1 (Left):
   key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6

DF2 (Right):
   key  data2
0   a      0
1   b      1
2   d      2


**Many-to-One Join:** `df1` has multiple rows labeled `a` and `b`, whereas `df2` has only one row for each value in the key column `key`.

The default is an **Inner Join** (intersection of keys).

In [ ]:
pd.merge(df1, df2)      # default: inner join

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


It is good practice to specify the key explicitly:

In [ ]:
# clearer with on. 'key' available on BOTH df
pd.merge(df1, df2, on="key") 

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


If the column names are different in each object, you can specify them separately using `left_on` and `right_on`:

In [130]:
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"], 
                    "data1": pd.Series(range(7), dtype="Int64")})

df4 = pd.DataFrame({"rkey": ["a", "b", "d"], 
                    "data2": pd.Series(range(3), dtype="Int64")})

print(df3)
print(df4)
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

  lkey  data1
0    b      0
1    b      1
2    a      2
3    c      3
4    a      4
5    a      5
6    b      6
  rkey  data2
0    a      0
1    b      1
2    d      2


,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


### Join Types (Inner, Outer, Left, Right)

You can specify the other options via the `how` parameter.

In [37]:
# Outer Join: Union of keys. Fills missing with NaN
pd.merge(df1, df2, how="outer")

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0
6,c,3,<NA>
7,d,<NA>,2


In [ ]:
# Outer Join with mismatched key names (=full outer join)
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how="outer")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


**Many-to-Many Join:**

In [131]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"], 
                    "data1": pd.Series(range(6), dtype="Int64")})

df2 = pd.DataFrame({"key": ["a", "b", "a", "b", "d"], 
                    "data2": pd.Series(range(5), dtype="Int64")})
print(df1)
print(df2)
pd.merge(df1, df2, how="inner")

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   b      5
  key  data2
0   a      0
1   b      1
2   a      2
3   b      3
4   d      4


,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


**Student Exercise:** Merge `df1` and `df2` with a left join.

In [132]:
pd.merge(df1, df2, how='left')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,c,3,<NA>
7,a,4,0
8,a,4,2
9,b,5,1


### Merging on Multiple Keys & Suffixes

To merge with multiple keys, pass a list of column names:

In [ ]:
left = pd.DataFrame({"key1": ["foo", "foo", "bar"], 
                     "key2": ["one", "two", "one"],
                     "lval": pd.Series([1, 2, 3], dtype='Int64')})

right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                      "key2": ["one", "one", "one", "two"],
                      "rval": pd.Series([4, 5, 6, 7], dtype='Int64')})

print(left)
print(right)
pd.merge(left, right, on=["key1", "key2"], how="outer")     # merge on >1 keys

  key1 key2  lval
0  foo  one     1
1  foo  two     2
2  bar  one     3
  key1 key2  rval
0  foo  one     4
1  foo  one     5
2  bar  one     6
3  bar  two     7


,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,foo,two,2,<NA>
3,bar,one,3,6
4,bar,two,<NA>,7


If there are overlapping non-key column names, `merge` adds suffixes `_x` and `_y` by default. You can customize this:

In [41]:
pd.merge(left, right, on="key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [42]:
pd.merge(left, right, on="key1", suffixes=("_left", "_right"))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### Merging on Index

If the merge key(s) is in the index, you can pass `left_index=True` or `right_index=True`.

In [134]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                      "value": pd.Series(range(6), dtype="Int64")})

right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])

print(left1)
print(right1)
pd.merge(left1, right1, left_on="key", right_index=True)

  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5
   group_val
a        3.5
b        7.0


,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


DataFrame has a `join` method which performs a left join by default. It's a convenient shortcut for index-on-index merging.

In [ ]:
left1.join(right1, on='key')    # left join for index-on-index

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


## 2.2 Reshaping and Pivoting

We often need to switch between **Wide Format** (Excel style, years as columns) and **Long Format** (Database style, one row per observation).

### Melt (Wide to Long)
Let's look at our stock price data. It is currently **Wide**.

In [45]:
# Reset index so Date is a column
price_reindex = price.reset_index()
price_reindex.head()

,Date,AAPL,GOOG,IBM,MSFT
0,2010-01-04,27.990226,313.062468,113.304536,25.884104
1,2010-01-05,28.038618,311.683844,111.935822,25.892466
2,2010-01-06,27.592626,303.826685,111.208683,25.733566
3,2010-01-07,27.541619,296.753749,110.823732,25.465944
4,2010-01-08,27.724725,300.709808,111.935822,25.641571


In [46]:
# Melt into Long format
melted = pd.melt(price_reindex, id_vars="Date")
melted.head()

,Date,variable,value
0,2010-01-04,AAPL,27.990226
1,2010-01-05,AAPL,28.038618
2,2010-01-06,AAPL,27.592626
3,2010-01-07,AAPL,27.541619
4,2010-01-08,AAPL,27.724725


**Student Exercise:** Rerun `melt` and pass arguments such that the new columns are named `Company` and `Price` respectively.

In [137]:
pd.melt(price_reindex, id_vars="Date", var_name='Company', value_name='Price')


,Date,Company,Price
0,2010-01-04,AAPL,27.990226
1,2010-01-05,AAPL,28.038618
2,2010-01-06,AAPL,27.592626
3,2010-01-07,AAPL,27.541619
4,2010-01-08,AAPL,27.724725
...,...,...,...
8565,2016-10-17,week_of_year,42
8566,2016-10-18,week_of_year,42
8567,2016-10-19,week_of_year,42
8568,2016-10-20,week_of_year,42


### Pivot (Long to Wide)
Using `pivot`, we can reshape back to the original layout:

In [139]:
print(melted.head())
reshaped = melted.pivot(index='Date', columns='variable', values='value')
reshaped.head()

        Date variable      value
0 2010-01-04     AAPL  27.990226
1 2010-01-05     AAPL  28.038618
2 2010-01-06     AAPL  27.592626
3 2010-01-07     AAPL  27.541619
4 2010-01-08     AAPL  27.724725


variable,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-04,27.990226,313.062468,113.304536,25.884104
2010-01-05,28.038618,311.683844,111.935822,25.892466
2010-01-06,27.592626,303.826685,111.208683,25.733566
2010-01-07,27.541619,296.753749,110.823732,25.465944
2010-01-08,27.724725,300.709808,111.935822,25.641571


---
# Part 3: Aggregation & Reporting

**Learning Objective:** Summarize data using GroupBy, Custom Aggregations, and Pivot Tables to answer business questions.

## 3.1 Data Aggregation (GroupBy)

Data aggregation is the process of grouping data together and performing calculations on them.

In [142]:
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None], 
                   "key2" : pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
                   "data1" : np.random.standard_normal(7), 
                   "data2" : np.random.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,-0.518849,-0.029009
1,a,2,-0.502963,1.249342
2,None,1,1.887443,-1.101367
3,b,2,-0.049025,-0.335023
4,b,1,0.384688,-0.174312
5,a,<NA>,0.822450,-1.308650
6,None,1,1.331975,-0.768148


If you want to compute the mean for each unique value in `key1`:

In [49]:
df.groupby("key1").mean()

,key2,data1,data2
key1,,,
a,1.5,-0.574496,0.18603
b,1.5,-0.149827,1.77288


It does not make sense to compute the mean for `key2` since it is a categorical variable and also serves as a key.

We can select the numeric columns to compute the mean for (after the `groupby` method):

In [50]:
df.groupby("key1")[["data1", "data2"]].mean()

,data1,data2
key1,,
a,-0.574496,0.18603
b,-0.149827,1.77288


Note that the following also works, since the returned result is a DataFrame, however it is less efficient as the selection/subset happens after the computation.

In [ ]:
df.groupby("key1").mean()[["data1", "data2"]]   #less efficient

,data1,data2
key1,,
a,-0.574496,0.18603
b,-0.149827,1.77288


You can group by more than 1 column. There is a useful GroupBy method `size` which returns a Series containing group sizes.

In [ ]:
df.groupby(['key1', 'key2']).size()     # size(): no. of rows

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

You can also group by other `Series`/`array`/`list` with the same length:

In [145]:
states = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]

print(df)
print(states)
print(years)
df["data1"].groupby([states, years]).mean()

   key1  key2     data1     data2
0     a     1 -0.518849 -0.029009
1     a     2 -0.502963  1.249342
2  None     1  1.887443 -1.101367
3     b     2 -0.049025 -0.335023
4     b     1  0.384688 -0.174312
5     a  <NA>  0.822450 -1.308650
6  None     1  1.331975 -0.768148
['OH' 'CA' 'CA' 'OH' 'OH' 'CA' 'OH']
[2005, 2005, 2006, 2005, 2006, 2005, 2006]


CA  2005    0.159743
    2006    1.887443
OH  2005   -0.283937
    2006    0.858332
Name: data1, dtype: float64

**Student Exercise:** Group by `key1` and `key2` and compute the standard deviation.

In [ ]:
#df.groupby(['key1', 'key2']).std()      # returns all NaN bcos need at least 2 value to compute std
df.groupby(['key1']).std()

,key2,data1,data2
key1,,,
a,0.707107,0.769855,1.278996
b,0.707107,0.306682,0.113640


## 3.2 Custom Aggregation

To use your own aggregation functions, pass any function that aggregates an array to the `aggregate` method or its short alias `agg`:

In [54]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [148]:
grouped = df.groupby("key1")
print(grouped)
grouped.agg(peak_to_peak)       #.agg use after groupby

,key2,data1,data2
key1,,,
a,1,1.341299,2.557991
b,1,0.433714,0.160711


You can pass a list of functions, or function names (for built-in functions) to `aggregate`: 

In [56]:
grouped.agg([peak_to_peak, "mean", "std"])

key2                       data1                      \
     peak_to_peak mean       std peak_to_peak      mean       std   
key1                                                                
a               1  1.5  0.707107     3.184194 -0.574496  1.661601   
b               1  1.5  0.707107     0.331000 -0.149827  0.234052   

            data2                     
     peak_to_peak     mean       std  
key1                                  
a        0.865577  0.18603  0.433357  
b        0.864672  1.77288  0.611415

## 3.3 Pivot Tables

Pivot tables are used to summarize, sort, reorganize, group, count, total or average data. It allows its users to transform columns into rows and rows into columns.

We will use the `tips.csv` dataset.

In [57]:
tips = pd.read_csv("../data/tips.csv")

# add a column with the tip percentage
tips["tip_pct"] = tips["tip"] / tips["total_bill"]

tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


The default aggregation for `pivot_table` is mean.

In [ ]:
# by default - mean values
tips.pivot_table(index=["day", "smoker"], values=["size", "tip", "tip_pct", "total_bill"])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

You can put `smoker` in the table columns and `time` and `day` in the rows:

In [ ]:
# add columns
tips.pivot_table(index=["time", "day"], columns="smoker", 
                 values=["tip_pct", "size"])

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

Add partial totals by passing `margins=True`:

In [60]:
tips.pivot_table(index=["time", "day"], columns="smoker", 
                 values=["tip_pct", "size"], margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

To use other aggregation functions, pass it to the `aggfunc` keyword:

In [ ]:
# by default, average. Use aggfunc to define. aggfunc=len -> no. of data pts
tips.pivot_table(index=["time", "smoker"], columns="day", 
                 values="tip_pct", aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

Use `fill_value` to fill missing values:

In [62]:
tips.pivot_table(index=["time", "smoker"], columns="day", 
                 values="tip_pct", aggfunc=len, margins=True, fill_value=0)

day            Fri  Sat  Sun  Thur  All
time   smoker                          
Dinner No        3   45   57     1  106
       Yes       9   42   19     0   70
Lunch  No        1    0    0    44   45
       Yes       6    0    0    17   23
All             19   87   76    62  244

**Student Exercise:** Compute the sum of `tip` in a pivot table with `day` and `time` in the rows and `smoker` in the column.

In [149]:
tips.pivot_table(index=["day", "time"], columns="smoker", 
                 values="tip", aggfunc=sum, margins=True)

smoker           No     Yes     All
day  time                          
Fri  Dinner    8.25   27.03   35.28
     Lunch     3.00   13.68   16.68
Sat  Dinner  139.63  120.77  260.40
Sun  Dinner  180.57   66.82  247.39
Thur Dinner    3.00     NaN    3.00
     Lunch   117.32   51.51  168.83
All          451.77  279.81  731.58

### Cross-Tabulation

A _cross-tabulation_ or _crosstab_ is a special case of pivot table that computes group frequencies (counts):

In [63]:
pd.crosstab(index=[tips["time"], tips["day"]], columns=tips["smoker"], margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

---
# Part 4: Advanced Toolkit (Optional / Deep Dive)

**Learning Objective:** Master complex data structures and advanced transformations. This section covers Hierarchical Indexing, Stacking, Concatenation, and custom Apply/Transform logic.

## 4.1 Hierarchical Indexing (MultiIndex)

Hierarchical indexing (MultiIndex) allows you to have multiple (two or more) _index levels_ on an axis. It enables "higher dimensional" data in a lower dimensional data structure.

In [64]:
data = pd.Series(np.random.uniform(size=9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                 [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.608849
   2    0.994688
   3    0.257343
b  1    0.671600
   3    0.161711
c  1    0.840970
   2    0.455615
d  2    0.077456
   3    0.604934
dtype: float64

In [65]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

You can use _partial indexing_ to select subsets of data:

In [66]:
data["b"]

1    0.671600
3    0.161711
dtype: float64

In [67]:
data["b":"c"]

b  1    0.671600
   3    0.161711
c  1    0.840970
   2    0.455615
dtype: float64

In [68]:
data.loc[["b", "d"]]

b  1    0.671600
   3    0.161711
d  2    0.077456
   3    0.604934
dtype: float64

You can also select from "inner" level:

In [69]:
data.loc[:, 2]

a    0.994688
c    0.455615
d    0.077456
dtype: float64

Hierarchical indexing works on both axes.

In [155]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                        index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                        columns=[['Ohio', 'Ohio', 'Colorado'],
                        ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

Setting names on the axes work as usual:

In [157]:
frame.index.names = ["key1", "key2"]
frame.columns.names = ["state", "color"]
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [158]:
frame.index.nlevels

2

Partial indexing works on columns too:

In [159]:
frame["Ohio"]

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

### Reordering and Sorting Levels

You may need to rearrange the order of the levels on an axis. The `swaplevel` method will swap the levels. The default is to swap the levels on the rows:

In [74]:
frame.swaplevel()

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [160]:
# swaps level 0 (outermost level) with level 1 (next level down) for the columns
frame.swaplevel(0, 1, axis=1)

color     Green  Red    Green
state      Ohio Ohio Colorado
key1 key2                    
a    1        0    1        2
     2        3    4        5
b    1        6    7        8
     2        9   10       11

You can also sort by a single level or subset of levels:

In [161]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

> **Student Exercise:** Swap the levels on the rows then sort the index by level `0`.

### Setting and Resetting Index

It's common to use one or more columns from a DataFrame as the row index.

In [77]:
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1), 
                      "c": ["one", "one", "one", "two", "two", "two", "two"], 
                      "d": [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


`set_index` will return a new DataFrame using one or more of its columns as the index.

In [78]:
frame2 = frame.set_index(["c", "d"])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

`reset_index` does the opposite of `set_index` and turns the index back into a column.

In [79]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


You can choose to drop the columns when resetting index:

In [80]:
frame2.reset_index(drop=True)

,a,b
0,0,7
1,1,6
2,2,5
3,3,4
4,4,3
5,5,2
6,6,1


## 4.2 Concatenation

You can join DataFrames along any axis which is referred to as _concatenation_ or _stacking_. This is akin to database `union` operations.

In [81]:
s1 = pd.Series([0, 1], index=["a", "b"], dtype="Int64")
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"], dtype="Int64")
s3 = pd.Series([5, 6], index=["f", "g"], dtype="Int64")

In [ ]:
pd.concat([s1, s2, s3])     # vertical stack, concat default outer join

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: Int64

By default, `concat` works along `axis="index"`, producing another Series. If you pass `axis="columns"`, the result will instead be a DataFrame:

In [83]:
pd.concat([s1, s2, s3], axis="columns")

,0,1,2
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


The default behavior of `concat` is union (`outer` join) of the indexes, you can also intersect them by passing `join='inner'`:

In [84]:
s4 = pd.concat([s1, s3])
pd.concat([s1, s4], axis="columns", join="inner")

,0,1
a,0,0
b,1,1


When combining Series along axis="columns", pass the `keys` argument for the DataFrame column headers:

In [85]:
pd.concat([s1, s2, s3], axis="columns", keys=["one", "two", "three"])

,one,two,three
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


**Student Exercise:** Concat `s1`, `s2` and `s3` along index and pass `keys=["one", "two", "three"]`.

If the index does not contain any relevant data, and you want to avoid concatenating based on indexes, you can pass the `ignore_index=True` argument:

In [86]:
df1 = pd.DataFrame(np.random.standard_normal((3, 4)), 
                   columns=["a", "b", "c", "d"])

df2 = pd.DataFrame(np.random.standard_normal((2, 3)), 
                   columns=["b", "d", "a"])

pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,0.075700,-1.075523,-3.196912,-0.980126
1,-1.278245,0.516962,-0.652370,1.131147
2,0.525349,-0.558584,-0.817125,1.582010
3,0.082436,0.692614,NaN,1.013267
4,-2.706789,-0.768313,NaN,-0.243281


## 4.3 Stacking and Unstacking

These are alternative reshaping methods to Melt/Pivot that work specifically on the Index levels.

In [87]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)), 
                    index=pd.Index(["Ohio", "Colorado"], name="state"),
                    columns=pd.Index(["one", "two", "three"], name="number"))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


The `stack` method pivots the columns into rows, producing a Series with a MultiIndex.

In [88]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

From a hierarchically indexed Series, you can rearrange the data back into a DataFrame with `unstack` , which pivots rows into columns.

In [89]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


You can unstack a different level by passing a level number or name:

In [90]:
result.unstack(level=0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


## 4.4 Advanced GroupBy: Apply

The most general-purpose GroupBy method is `apply`, which splits the object being manipulated into pieces, invokes the passed function on each piece, and then concatenates the pieces.

Suppose we want to select the top five `tip_pct` values by group. First, write a function that selects the rows with the largest values in a particular column:

In [91]:
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]

In [92]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


We can then `apply` this function by different groups using `groupby`:

In [93]:
tips.groupby("smoker").apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

You can pass the arguments to the function as follows:

In [94]:
tips.groupby(["smoker", "day"]).apply(top, n=2, column="total_bill")

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
            91        22.49   3.50     No   Fri  Dinner     2  0.155625
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
            59        48.27   6.73     No   Sat  Dinner     4  0.139424
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
            112       38.07   4.00     No   Sun  Dinner     3  0.105070
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
            85        34.83   5.17     No  Thur   Lunch     4  0.148435
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
            90        28.97   3.00    Yes   Fri  Dinner     2  0.103555
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
            102       44.30   2.50    Yes   Sat  Dinner     3  0.056433
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
            184       40.55   3.00    Yes   Sun  Dinner     2  0.073983
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982
            83        32.68   5.00    Yes  Thur   Lunch     2  0.152999

**Student Exercise:** Apply the function on `day` and `time` group.

## 4.5 Advanced GroupBy: Transform

You can also transform your data using the `transform` method. It is similar to `apply` but the function must:
- Produce a scalar value to be broadcast to the shape of the group chunk, or
- Return an object that is the same shape as the group chunk

This is useful for z-score normalization within groups.

In [95]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4, 'value': np.arange(12.)})
g = df.groupby('key')['value']
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

`transform` produce a Series of the same shape as `df['value']` but with values replaced by the average grouped by `key`.

In [96]:
g.transform(lambda g: g.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [97]:
def normalize(x):
    return (x - x.mean()) / x.std()

g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

### Post-class

In [19]:
'''
Create a dummy dataframe (or load one) with an index of daily dates for the year 2023.
Generate random "Sales" data for each day.
Resample this daily data to find the Total (Sum) Sales per Month.
'''

dates = pd.date_range("2023-01-01", periods=365)
sales = np.random.standard_normal(365)
df1 = pd.DataFrame(data = {'sales':sales} ,index=dates)
print(df1.head())

df1.resample('M').sum()

               sales
2023-01-01  0.135042
2023-01-02  0.850980
2023-01-03  0.698577
2023-01-04 -0.506010
2023-01-05 -1.681719


,sales
2023-01-31,-3.576588
2023-02-28,4.310807
2023-03-31,0.958251
2023-04-30,-4.722786
2023-05-31,-2.085913
2023-06-30,5.389928
2023-07-31,-6.463159
2023-08-31,-5.663059
2023-09-30,-3.649773
2023-10-31,4.111077


In [22]:
'''
Create a small dataframe with columns: ['Date', 'Region', 'Sales']. 
Populate it with data for 3 regions (North, South, East) over 5 different dates.
Use pivot() to reshape this data so that: 
Date is the Index.
Region names are the Columns (North, South, East).
Sales are the Values.
'''

dates2 = [datetime(2026, 1, 5), datetime(2026, 1, 7), datetime(2026, 1, 8), datetime(2026, 1, 10), datetime(2026, 1, 12)]
sales2 = [100, 300, 600, 700, 888]
region2 = ['North', 'East', 'South', 'East', 'East']
df2 = pd.DataFrame({'Date':dates2, 'Region':region2, 'Sales':sales2})

df2.pivot(index='Date', columns='Region', values='Sales')

Region,East,North,South
Date,,,
2026-01-05,NaN,100.0,NaN
2026-01-07,300.0,NaN,NaN
2026-01-08,NaN,NaN,600.0
2026-01-10,700.0,NaN,NaN
2026-01-12,888.0,NaN,NaN


In [24]:
'''
Using the tips dataset (or similar), create a Pivot Table that shows:
Rows: Day of the Week.
Columns: Time (Lunch vs. Dinner).
Values: Total Bill.
Aggregation: Calculate the Average (mean) bill size.
Margins: Include the "All" row/column totals.
'''

tips = pd.read_csv("../data/tips.csv")
print(tips)

tips.pivot_table(index="day", columns="time", values="total_bill", aggfunc="mean", margins=True)


     total_bill   tip smoker   day    time  size
0         16.99  1.01     No   Sun  Dinner     2
1         10.34  1.66     No   Sun  Dinner     3
2         21.01  3.50     No   Sun  Dinner     3
3         23.68  3.31     No   Sun  Dinner     2
4         24.59  3.61     No   Sun  Dinner     4
..          ...   ...    ...   ...     ...   ...
239       29.03  5.92     No   Sat  Dinner     3
240       27.18  2.00    Yes   Sat  Dinner     2
241       22.67  2.00    Yes   Sat  Dinner     2
242       17.82  1.75     No   Sat  Dinner     2
243       18.78  3.00     No  Thur  Dinner     2

[244 rows x 6 columns]


time,Dinner,Lunch,All
day,,,
Fri,19.663333,12.845714,17.151579
Sat,20.441379,NaN,20.441379
Sun,21.410000,NaN,21.410000
Thur,18.780000,17.664754,17.682742
All,20.797159,17.168676,19.785943
